# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  1. Загрузка данных и изучение общей информации. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
# Общая информация о таблице
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [2]:
# Количество уникальных значений в столбце 'children'
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [3]:
# Диапазон значений по столбцу 'days_employed'
print(data['days_employed'].sort_values(ascending = False))

6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
             ...      
21489              NaN
21495              NaN
21497              NaN
21502              NaN
21510              NaN
Name: days_employed, Length: 21525, dtype: float64


In [4]:
# Подсчет нулевых значений в столбце 'days_employed'
print(data[data['days_employed'].isnull()].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


In [5]:
# Подсчет значений возраста, выходящих за разумный пределы
print(data.loc[(data['dob_years'] <= 16) | (data['dob_years'] >= 100)].count())

children            101
days_employed        91
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income         91
purpose             101
dtype: int64


In [6]:
# Возможные варианты образования
print(data[['education','education_id']].groupby('education').count())

                     education_id
education                        
ВЫСШЕЕ                        274
Высшее                        268
НАЧАЛЬНОЕ                      17
НЕОКОНЧЕННОЕ ВЫСШЕЕ            29
Начальное                      15
Неоконченное высшее            47
СРЕДНЕЕ                       772
Среднее                       711
УЧЕНАЯ СТЕПЕНЬ                  1
Ученая степень                  1
высшее                       4718
начальное                     250
неоконченное высшее           668
среднее                     13750
ученая степень                  4


In [7]:
# Возможные варианты семейного положения
print(data[['family_status','family_status_id']].groupby('family_status').count())

                       family_status_id
family_status                          
Не женат / не замужем              2813
в разводе                          1195
вдовец / вдова                      960
гражданский брак                   4177
женат / замужем                   12380


In [8]:
# Возможные варианты пола
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [9]:
# Возможные варианты типов дохода
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


In [10]:
# Возможные варианты просрочки
print(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


In [11]:
# Возможные варианты цели кредита
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Выводы по входным данным

1. Количество значений в столбцах не одинаково.В двух столбцах значений меньше, соответственно в них есть пропуски данных
2. Тип значений столбцов устраивает кроме debt - надо поставить булево значение (?)
3. Анализ столбцов
  1. children
      * Посмотрев на список уникальных значений можно сделать следующие предположения:
      * 20 детей конечно может быть, но очень вряд ли. Скорее всего случайно добавили 0. Надо заменить на 2
      * Аналогично с показателем -1, надо исправить на 1
  2. days_employed
      * Очень большой разброс данных. Есть и отрицательные числа и NaN и огромныйе числа типа 482 лет (дни /365). Есть гипотезы.
      * Отрицательные величины на самом деле положительные, у которых в начале появился "-". Надо взять их по модулю
      * Пустые значения означают или отсутствие рабочего стажа или отсутствие информации. Их 2174 штуки или 10%, что существенно. Если это отсутствие информации, то можно заполнить пропуски средним значением в зависимости от возраста. Если это отсутствие рабочего стажа, то заполнить 0. Надо узнать причину пропусков. Учитывая, что данные корелируют со столбцом дохода, большая вероятность, что данные по работе не были занесены и тогда эти строки или удалить или заполнить по среднему.
      * Большие значения стажа были введены не в днях, а в часах и их надо дополнительно разделить на 24
  3. dob_years
      * 101 запись меньше 16 лет и больше 100, а таким кредит не дают. Их всего 0,4% и их можно удалить
  4. education
      * Есть дубликаты значений, написанные в разном регистре. Надо привести к нижнему для категоризации
  5. education_id
      * Есть 5 значений, соответствующие 5 типам образавания. Вопрос насколько они корректны? Можно исходя из них заполнить столбец "education" и сравнить с тем что сделали выше
  6. family_status 
      * никаких претензий нет
  7. family_status_id
      * никаких претензий нет
  8. gender 
      * одна непонятная запись "XNA", ее можно удалить 
  9. income_type
      * 4 основных категрии, 4 - незначительных. Их можно или удалить, или свести в одну категоию "прочие"
  10. debt
      * никаких претензий нет 
  11. total_income
      * Для удобства восприятия возможно отформатировать до 0 знаков после запятой. 
  12. purpose
      * Много дубликатов, связанны, видимо, со свободной формой заполнения. Надо провести леммизацию и определить категории

## 2. Предобработка данных

### Обработка пропусков

In [12]:
# Приводим в порядок столбец 'children', заменяя значения
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
# Сокращаем количество категорий в столбце 'income_type'
data.loc[data['income_type'] == 'безработный', 'income_type'] = 'прочее'
data.loc[data['income_type'] == 'предприниматель', 'income_type'] = 'прочее'
data.loc[data['income_type'] == 'в декрете', 'income_type'] = 'прочее'
data.loc[data['income_type'] == 'студент', 'income_type'] = 'прочее'
# Функция вычисления медианного дохода по категориям
def median_income_type(income_type_in):
    data.loc[data['income_type'] == income_type_in]
    median_income_type = data['total_income'].median()
    return median_income_type
# Заполнение пропусков медианным доходом по категориям
data.loc[data['income_type'] == 'сотрудник','total_income'] = data.loc[data['income_type'] == 'сотрудник','total_income'].fillna(value = median_income_type('сотрудник'))
data.loc[data['income_type'] == 'пенсионер','total_income'] = data.loc[data['income_type'] == 'пенсионер','total_income'].fillna(value = median_income_type('пенсионер'))
data.loc[data['income_type'] == 'прочее','total_income'] = data.loc[data['income_type'] == 'прочее','total_income'].fillna(value = median_income_type('прочее'))
data.loc[data['income_type'] == 'компаньон','total_income'] = data.loc[data['income_type'] == 'компаньон','total_income'].fillna(value = median_income_type('компаньон'))
data.loc[data['income_type'] == 'госслужащий','total_income'] = data.loc[data['income_type'] == 'госслужащий','total_income'].fillna(value = median_income_type('госслужащий'))

### Замена типа данных

In [13]:
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

In [14]:
# Приведение столбца профессий к нижнему регистру
data['education'] = data['education'].str.lower()

In [15]:
# Удаляем дубликаты
data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())

0


### Лемматизация

In [16]:
# импорт библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem()
# Функция лемматизации
def purpose_lem(row):
    lemmas = ' '.join(m.lemmatize(row))
    return lemmas
# Запись лемм в новый столбец 'purpose_lem'
data['purpose_lem'] = data['purpose'].apply(purpose_lem)
# импорт контейнера для подсчёта упоминаний слов
from collections import Counter
# вывод количества уникальных лемм 
print(Counter(' '.join(data['purpose_lem'].tolist()).split()))

Counter({'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


### Категоризация данных

Для ответов на вопросы нам понадобятся категории в столбцах:
1. Дети
    * 0 (нет детей)
    * 1-2 (есть дети)
    * 3 и больше (многодетные)
2. Семейное положение
    * женат / замужем
    * гражданский брак
    * Не женат / не замужем
    * в разводе
    * вдовец / вдова  
3. Уровень дохода
    * Низкий доход (до 20 000)
    * Средний доход (от 20 000 до 50 000)
    * Высокий доход (более 50 000)
4. Цели
    * Недвижимость / жилье / ремонт
    * Автомобиль
    * Образование
    * Свадьба

In [17]:
# Функция определения категории детей
def children_category(amount):
    if amount == 0:
        return 'нет детей'
    if amount >= 3:
        return 'многодетные'
    return 'есть дети'
# Новый столбец с категориями по детям
data['children_category'] = data['children'].apply(children_category)

# Функция определения категории дохода
def total_income_category (amount):
    if amount <= 100000:
        return 'Низкий доход'
    if amount >= 200000:
        return 'Высокий доход'
    return 'Средний доход'
# Новый столбец с категориями по категориям дохода
data['total_income_category'] = data['total_income'].apply(total_income_category)

# Функция определения категории цели кредита
def purpose_catrgory (row):
    for word in row.split(' '):
        if word == 'недвижимость':
            return 'недвижимость'
        if word == 'жилье':
            return 'недвижимость'        
        if word == 'ремонт':
            return 'недвижимость'
        if word == 'автомобиль':
            return 'автомобиль' 
        if word == 'образование':
            return 'образование'  
        if word == 'свадьба':
            return 'свадьба'  
# Запись категорий в новый столбец 'purpose_catrgory'
data['purpose_catrgory'] = data['purpose_lem'].apply(purpose_catrgory)       

##  3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
# Подсчет долей возврата кредита по категориям детей
groupby_children_category = data.groupby('children_category')['debt'].sum() / data.groupby('children_category')['debt'].count()
print(groupby_children_category.sort_values()*100)


children_category
нет детей      7.543822
многодетные    8.157895
есть дети      9.265359
Name: debt, dtype: float64


### Вывод 1
Лучше всего возвращают кредиты заемщики без детей.  
Однако зависимость между наличием детей и возвратом кредита не велика и составляет 1,72%. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
groupby_family_status = data.groupby('family_status')['debt'].sum() / data.groupby('family_status')['debt'].count()
print(groupby_family_status.sort_values())

family_status
вдовец / вдова           0.065693
в разводе                0.071130
женат / замужем          0.075452
гражданский брак         0.093471
Не женат / не замужем    0.097509
Name: debt, dtype: float64


### Вывод 2
В целом, здесь видно, что наиболее надежные заемщики, это те, кто находится или когда-либо был в официальном браке.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [20]:
# Подсчет долей возврата кредита по категориям дохода
groupby_income_category = data.groupby('total_income_category')['debt'].sum() / data.groupby('total_income_category')['debt'].count()
print(groupby_income_category.sort_values())

total_income_category
Высокий доход    0.070667
Низкий доход     0.079319
Средний доход    0.086289
Name: debt, dtype: float64


### Вывод 3
Чем выше доход, тем меньше проблем с возратами кредита. 

Чем выше доход, тем меньше проблем с возратами кредита. Однако, что характерно, клиенты с самым нихким уровнем дохода отдают кредиты лучше, чем клиенты со средним.

### Как разные цели кредита влияют на его возврат в срок?

In [21]:
groupby_purpose = data.groupby('purpose_catrgory')['debt'].sum() / data.groupby('purpose_catrgory')['debt'].count()
print(groupby_purpose.sort_values())

purpose_catrgory
недвижимость    0.072334
свадьба         0.080034
образование     0.092200
автомобиль      0.093590
Name: debt, dtype: float64


### Вывод 4
Лучше всего возвращают кредиты на недвижимость, хуже всего - на аввтомобиль

### При оформлении кредита на разные цели заемщики какого семейного положения наболее аккуратны?

In [22]:
print(data.pivot_table('debt', index='purpose_catrgory', columns = 'family_status'))

family_status     Не женат / не замужем  в разводе  вдовец / вдова  \
purpose_catrgory                                                     
автомобиль                     0.128728   0.074733        0.091743   
недвижимость                   0.081454   0.069527        0.051661   
образование                    0.107452   0.071429        0.075377   
свадьба                             NaN        NaN             NaN   

family_status     гражданский брак  женат / замужем  
purpose_catrgory                                     
автомобиль                0.117512         0.083699  
недвижимость              0.092012         0.069349  
образование               0.148515         0.083237  
свадьба                   0.080034              NaN  


### Вывод 5
При цели кредита "на автомобиль" лучше всего возвращают заемщики, находящиеся в разводе, хуже всего - не женатые  
При цели кредита "на недвижимость" лучше всего возвращают заемщики в статусе "вдовец/вдова", хуже всего - в гражданском браке   При цели кредита "на образование" лучше всего возвращают заемщики в статусе "женат/замужем", хуже всего - в гражданском браке   Кредит "на свадьбу" берут только заемщики в гражданском браке.

## 4. Общий вывод

### Анализ предоставленных данных показал:
1. Между наличием детей и возвратом кредита в срок зависимости практически нет. Лучше всего возвращают кредиты заемщики без детей.
2. Между семейным положением и возвратом кредита в срок зависимомь минимальна. Наиболее аккуратные - "вдовец / вдова", наиболее проблемные - без семьи
3. Наиболее аккуратно платят заемщики с высоким доходом, а хуже  - со средним доходом.
4. Лучше всего возвращают кредиты на автомобиль, хуже всего - на недвижимость
5. При цели кредита "на автомобиль" лучше всего возвращают заемщики, находящиеся в разводе, хуже всего - не женатые  
При цели кредита "на недвижимость" лучше всего возвращают заемщики в статусе "вдовец/вдова", хуже всего - в гражданском браке   При цели кредита "на образование" лучше всего возвращают заемщики в статусе "женат/замужем", хуже всего - в гражданском браке   Кредит "на свадьбу" берут только заемщики в гражданском браке.  

### Рекомендации к дальнейшему сбору данных:
1. Уточнить указание к заполнении "Общего трудового стажа", например месяцы и сделать предварительную проверку на адекватность значения
2. При заполнении сделать категории типов занятости.
3. При заполнении сделать категории целей кредита.

### Дополнительно:
Возможно имеет смысл рассчитать кросс-категорийные показатели. Например: сумма кредита и доход заемщика, цели кредита и семейное положение и т.д.